## 数据预处理

In [55]:
# -*- coding: UTF-8 -*-

"""
@author: wty-yy
@software: Jupyter
@file: prepare.py
@time: 2022/11/14 15:03
"""

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import collections
import re
from tqdm import tqdm
from nltk import word_tokenize  # 分词
from nltk.corpus import stopwords  # 停用词
from nltk.stem.porter import PorterStemmer  # 使用Porter词干提取方法
from nltk.stem import WordNetLemmatizer  # 词性还原

In [130]:
def extractWords(words):  # 提取分词
    words = words.lower()
    words = word_tokenize(words)  # 分词
    dropWords = ["n't"]  # 这个是计算结果中出现次数第一的，但明显不重要
    words = [word for word in words if re.match(r'[A-Za-z]', word) and word not in dropWords]  # 保证单词中必须包含字母
    stops = set(stopwords.words('english'))
    words = [word for word in words if word not in stops]
    tmp = []  # 词干提取+还原词性
    for word in words:
        stem = PorterStemmer().stem(word)  # 词干提取
        pos = ['n', 'v', 'a', 'r', 's']  # 名词，动词，形容词，副词，附属形容词
        for p in pos:
            stem = WordNetLemmatizer().lemmatize(stem, pos=p)
        tmp.append(stem)  # 还原词性，附属形容词
    words = tmp
    return words

def initDataset(fname, showInfo=True):
    path = Path(fname)
    folds = [f.name for f in path.iterdir() if f.is_dir()]
    allwords, words, fileNum, counter = [[] for _ in range(20)], [[] for _ in range(20)], [], []
    for id, fold in enumerate(folds):
        print(f'处理第{id+1}/{len(folds)}个文件夹{fold}中...')
        now = path.joinpath(fold)
        files = [f.name for f in now.iterdir() if f.is_file()]
        for file in tqdm(files):
            pathFile = now.joinpath(file)
            with open(pathFile, errors='replace') as f:
                s = f.readline()
                while s != "\n":  # 先找到第一个换行符，下面则是正文
                    s = f.readline()
                text = f.read()
            allwords[id] += extractWords(text)
            words[id].append(extractWords(text))
        fileNum.append(len(files))  # 记录文件数目
        counter.append(collections.Counter(allwords[id]))  # 统计词频
        
    totalCounter = collections.Counter([w for word in allwords for w in word])
    if showInfo:
        print(f'{"Class":>25} {"Id":>5} {"Files":>8} {"Words":>8}  {"Most common words"}')  # 显示文件相关信息
        for i, fold in enumerate(folds):
            print(f'{fold:>25}:{i:>5} {fileNum[i]:>8} {len(counter[i]):>8}  {[t[0] for t in counter[i].most_common(5)]}')
        print(f'{"":>25} {"":>5} {sum(fileNum):>8} {len(totalCounter):>8}  {[t[0] for t in totalCounter.most_common(5)]}')
    
    return words, totalCounter


In [131]:
print(WordNetLemmatizer().lemmatize('get', pos='v'), WordNetLemmatizer().lemmatize('got', pos='v'), WordNetLemmatizer().lemmatize('gotten', pos='v'))

get get get


In [132]:
[1,1]+[2,2]

[1, 1, 2, 2]

In [133]:
words, totalCounter = initDataset(r'D:\yy\program\NLP\hw2\20_newsgroups')

处理第1/20个文件夹alt.atheism中...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 64.07it/s]


处理第2/20个文件夹comp.graphics中...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 76.84it/s]


处理第3/20个文件夹comp.os.ms-windows.misc中...


...

处理第19/20个文件夹talk.politics.misc中...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 51.86it/s]


处理第20/20个文件夹talk.religion.misc中...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 63.23it/s]


                    Class    Id    Files    Words  Most common words
              alt.atheism:    0     1000    10950  ['write', 'say', 'one', 'god', 'would']
            comp.graphics:    1     1000    13406  ['imag', 'file', 'use', 'program', 'write']
  comp.os.ms-windows.misc:    2     1000    48850  ['max', 'g', 'r', 'q', 'p']
 comp.sys.ibm.pc.hardware:    3     1000    10353  ['drive', 'use', 'get', 'card', 'scsi']
    comp.sys.mac.hardware:    4     1000     9354  ['use', 'mac', 'get', 'write', 'appl']
           comp.windows.x:    5     1000    20392  ['x', 'use', 'window', 'file', 'program']
             misc.forsale:    6     1000    10830  ['new', 'sale', 'offer', 'use', 'sell']
                rec.autos:    7     1000    10378  ['car', 'write', 'get', 'articl', 'would']
          rec.motorcycles:    8     1000    10207  ['write', 'bike', 'get', 'articl', 'dod']
       rec.sport.baseball:    9     1000     9164  ['game', 'year', 'write', 'good', 'get']
         rec.sport.hoc

```
                    Class  Id Files  Words  Most common words
              alt.atheism:  0  1000  10950  ['write', 'say', 'one', 'god', 'would']
            comp.graphics:  1  1000  13406  ['imag', 'file', 'use', 'program', 'write']
  comp.os.ms-windows.misc:  2  1000  48850  ['max', 'g', 'r', 'q', 'p']
 comp.sys.ibm.pc.hardware:  3  1000  10353  ['drive', 'use', 'get', 'card', 'scsi']
    comp.sys.mac.hardware:  4  1000   9354  ['use', 'mac', 'get', 'write', 'appl']
           comp.windows.x:  5  1000  20392  ['x', 'use', 'window', 'file', 'program']
             misc.forsale:  6  1000  10830  ['new', 'sale', 'offer', 'use', 'sell']
                rec.autos:  7  1000  10378  ['car', 'write', 'get', 'articl', 'would']
          rec.motorcycles:  8  1000  10207  ['write', 'bike', 'get', 'articl', 'dod']
       rec.sport.baseball:  9  1000   9164  ['game', 'year', 'write', 'good', 'get']
         rec.sport.hockey: 10  1000  11311  ['game', 'team', 'play', 'go', 'get']
                sci.crypt: 11  1000  13087  ['key', 'use', 'encrypt', 'would', 'write']
          sci.electronics: 12  1000  10480  ['use', 'one', 'would', 'write', 'get']
                  sci.med: 13  1000  15271  ['use', 'one', 'write', 'get', 'articl']
                sci.space: 14  1000  13867  ['space', 'would', 'write', 'orbit', 'one']
   soc.religion.christian: 15   997  12616  ['god', 'christian', 'one', 'would', 'say']
       talk.politics.guns: 16  1000  14626  ['gun', 'would', 'write', 'peopl', 'articl']
    talk.politics.mideast: 17  1000  15105  ['armenian', 'say', 'peopl', 'one', 'write']
       talk.politics.misc: 18  1000  13727  ['would', 'write', 'peopl', 'say', 'articl']
       talk.religion.misc: 19  1000  12390  ['write', 'say', 'one', 'god', 'would']
                              19997 146437  ['write', 'would', 'one', 'use', 'get']
```

## K近邻

选取前1000个单词作为词向量的维度.

In [134]:
N = 1000
value = list(totalCounter.values())
keyword = totalCounter.most_common(1000)

In [135]:
word2num, num2word = {}, {}
for i, word in enumerate(keyword):
    word2num[word[0]] = i
    num2word[i] = word
    print(word[0], end=', ')

write, would, one, use, get, articl, say, know, like, think, make, peopl, good, go, time, x, see, also, could, work, u, take, right, new, want, system, even, way, year, thing, come, well, find, may, give, look, need, god, problem, much, mani, tri, first, two, file, mean, max, believ, call, run, question, point, q, anyon, post, seem, program, state, window, tell, differ, r, drive, read, realli, someth, plea, includ, g, sinc, thank, number, p, ca, back, univers, still, govern, reason, help, inform, day, start, person, game, gener, part, follow, might, support, c, law, sure, last, long, ask, case, fact, never, do, let, interest, set, christian, must, without, possibl, hear, group, comput, power, anoth, someon, car, avail, lot, n, b, name, show, put, keep, key, imag, line, great, exist, chang, live, send, actual, word, world, control, place, claim, high, list, happen, probabl, anyth, etc, data, howev, around, book, w, littl, opinion, v, everi, bite, card, kill, true, consid, least, cours, 

### 构建词向量

归一化到半径为100的球上

In [164]:
def word2vector(word):
    x = np.ones(N)
    for t in w:
        if t in word2num:
            x[word2num[t]] += 1
    x /= x.sum() / 100
    return x
    
data = []  # (词向量, 类别)  共有19997个词向量
for i in range(20):
    for w in words[i]:
        x = word2vector(w)
        data.append((x, i))

In [201]:
test_words, _ = initDataset(r'D:\yy\program\NLP\hw2\mini_newsgroups')

处理第1/20个文件夹alt.atheism中...


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 69.90it/s]


处理第2/20个文件夹comp.graphics中...


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 44.73it/s]


处理第3/20个文件夹comp.os.ms-windows.misc中...


...

处理第19/20个文件夹talk.politics.misc中...


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 51.23it/s]


处理第20/20个文件夹talk.religion.misc中...


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 65.62it/s]


                    Class    Id    Files    Words  Most common words
              alt.atheism:    0      100     3096  ['write', 'say', 'articl', 'one', 'make']
            comp.graphics:    1      100     5273  ['imag', 'file', 'use', 'jpeg', 'format']
  comp.os.ms-windows.misc:    2      100     5038  ['p', 'r', 'g', 'window', 'w']
 comp.sys.ibm.pc.hardware:    3      100     2567  ['use', 'drive', 'would', 'one', 'work']
    comp.sys.mac.hardware:    4      100     2660  ['drive', 'problem', 'use', 'get', 'mac']
           comp.windows.x:    5      100     3715  ['x', 'window', 'use', 'entri', 'program']
             misc.forsale:    6      100     2669  ['new', 'plea', 'work', 'use', 'sale']
                rec.autos:    7      100     3177  ['car', 'write', 'get', 'one', 'like']
          rec.motorcycles:    8      100     3218  ['write', 'bike', 'articl', 'get', 'would']
       rec.sport.baseball:    9      100     2889  ['year', 'write', 'game', 'good', 'get']
         rec.spor

In [235]:
def KNN(word, K=5):
    now = word2vector(word)
    dist = []
    for x, y in data:
        dist.append((np.linalg.norm(now - x), y))
    dist = sorted(dist, key=(lambda x: x[0]))
    dist = dist[1:K+1]
    classify = [c[1] for c in dist]
    counter = collections.Counter(classify)
    return counter.most_common()[0][0]

In [233]:
K = [5]
rate = []
for k in K:
    right = 0
    for w in tqdm(test_words[1]):
        if KNN(w, k) == 1:
            right += 1
    rate.append(right/len(test_words[0]))
    print(f'K={k}, 正确率: {rate[-1]:%}')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.04it/s]

K=5, 正确率: 55.000000%


In [236]:
for i in range(20):
    right = 0
    for w in tqdm(test_words[i]):
        if KNN(w, k) == i:
            right += 1
    rate.append(right/len(test_words[i]))

for i in range(20):
    print(f'第{i+1}组类别，正确率: {rate[i]}')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.16it/s]

第1组类别，正确率: 0.55
第2组类别，正确率: 0.43
第3组类别，正确率: 0.55
第4组类别，正确率: 0.51
第5组类别，正确率: 0.38
第6组类别，正确率: 0.56
第7组类别，正确率: 0.44
第8组类别，正确率: 0.42
第9组类别，正确率: 0.46
第10组类别，正确率: 0.67
第11组类别，正确率: 0.57
第12组类别，正确率: 0.7
第13组类别，正确率: 0.62
第14组类别，正确率: 0.51
第15组类别，正确率: 0.57
第16组类别，正确率: 0.57
第17组类别，正确率: 0.5
第18组类别，正确率: 0.56
第19组类别，正确率: 0.72
第20组类别，正确率: 0.43
